In [1]:
@info("Loading packages")
using Pkg; Pkg.activate("../test")
using Plots, StatsPlots, Plots.Measures
using StatsBase
using Glob
using DataFrames, DataFramesMeta
using ProgressMeter
default(leg = false, lw = 1.5)

using ComponentArrays, StaticArrays
using Test
using Parameters
using Distributions
using OrdinaryDiffEq

# establishing type hierarchy
abstract type AbstractParams end
abstract type AbstractParamCollection end # an AbstractParamCollection contain a defined set of multiple AbstractParams instances
abstract type AbstractSpeciesParams <: AbstractParams end
abstract type AbstractGlobalODEParams <: AbstractParams end
abstract type AbstractABM end
abstract type AbstractAgent end

include("../src/DoseResponse.jl")
include("../src/Params.jl")
include("../src/Solvers.jl")
include("../src/StateVars.jl")
include("../src/IO.jl")
include("../src/ModelFunctions.jl")
include("../src/DEBODE.jl")
include("../src/ImpliedTraits.jl")
include("../src/Macros.jl")

┌ Info: Loading packages
└ @ Main c:\Users\simon\Documents\Julia\Hydra\test\abm.ipynb:1
  Activating project at `c:\Users\simon\Documents\Julia\Hydra\test`


@replicates

## ABM

In [2]:
abstract type AbstractABM end

"""
Currently assumes that all agents are represented by a single agent type `AgentType`.
"""
mutable struct ABM <: AbstractABM
    p::AbstractParamCollection # parameter collection
    t::Float64 # current simulation time
    dt::Float64 # timestep
    euler::Function # definition of the euler function for the given dt
    saveat::Float64
    
    u::ComponentVector # global state variables
    du::ComponentVector # global derivatives

    AgentID_count::Int64 # cumulative number of agents in the simulation
    agents::AbstractVector # population of agents
    aout::AbstractVector # agent output
    mout::AbstractVector # model output

    """
    Instantiate ABM from param collection `p`. 
    """
    function ABM(
        p::A; 
        dt = 1/24, 
        saveat = 1,
        execute = true
        ) where A <: AbstractParamCollection

        abm = new() # initialize ABM object

        abm.p = p # store the parameters
        abm.t = 0. # initialize simulation time
        abm.dt = dt # timestep is given as keyword argument
        abm.euler = defeuler(dt) # Euler function for the given dt
        abm.saveat = saveat
        
        abm.u = init_substates_global(p) # initialize the global substates
        abm.du = similar(abm.u) # initialize the global derivatives
        abm.du.X_p = 0.
        abm.du.C_W .= 0.
        
        abm.AgentID_count = 0 # set agent count
        initialize_agents!(abm) # initailize the population of agents
        abm.aout = [] # initialize agent output
        abm.mout = [] # initialize model output

        if execute
            run!(abm) # run the model

            return prepare_output(abm) # prepare output in dataframes and return
        else
            return abm
        end
    end
end

ABM

In [3]:
using Random

"""
step!(
    abm::ABM; 
    )

Execution of a generic ABM step, following the schedule: 

1. Shuffle agents
2. Calculate global derivatives
3. Calculate agent derivatives
4. Update agent states
5. Record agent states
6. Update global states
7. Record global states

It is important that the agent steps occur between calculation of the global derivatives and 
updating the global states, because global derivatives which may be influenced by agent derivatives 
are initialized during calculation of the global states and may be mutated by the agent step. <br>
Changing this order will lead to erroneous computation of the interaction between agents and the environment.
"""
function step!(abm::ABM)

    du, u, p = abm.du, abm.u, abm.p
    t = abm.t

    shuffle!(abm.agents)

    for func! in abm.p.glb.odefuncs # execute global ODE-based functions
        func!(du, u, p, t)
    end

    for func! in abm.p.glb.rulefuncs # execute global rule-based functions
        func!(abm)
    end
    
    for a in abm.agents # for every agent
        step!(a, abm) # execute agent steps
        record!(a, abm) # record agent data
    end

    map!(abm.euler, u, du, u) # apply the euler scheme to global states
    record!(abm) # record global states
    filter!(a -> a.u.agn.dead == false, abm.agents) # remove dead agents

    abm.t += abm.dt
end


step!

In [4]:
"""
    init_agents!(abm::AbstractABM)::nothing
Initialize the population of 
"""
function initialize_agents!(abm::AbstractABM)::Nothing

    abm.agents = [] # initialize a vector of agents with undefined values and defined length

    for i in 1:abm.p.glb.N0 # for the number of initial agents
        
        push!(abm.agents, abm.p.glb.AgentType(abm)) # initialize an agent and add it to the vector of agents
    end

    return nothing
end

initialize_agents!

## Agent

In [5]:
abstract type AbstractAgent end
import Base:copy


"""
The definition of a generic AbstractAgent.
"""
mutable struct DEBAgent <: AbstractAgent
    p::AbstractParamCollection
    u::ComponentVector
    du::ComponentVector
    AgentID::Int64
    adata_t::AbstractVector

    """
        DEBAgent(abm::ABM)
    
    Initialization of a generic DEB agent within ABM structure.
    """
    function DEBAgent(abm::ABM)
        a = new()

        a.p = copy(abm.p) # parameters; TODO: #29 avoid copying everything
        a.p.agn = AgentParams(a.p.spc) # assign agent-level parameters (induces individual variability)
        initialize_statevars!(a, abm) # initialize agent-level state variables
        a.du = similar(a.u) # initialize agent-level derivatives
        a.du.glb = abm.du # reference to global derivatives
        a.du.agn = similar(a.u.agn) # derivatives of agent substates have same shape as states
        a.AgentID = abm.AgentID_count # assign AgentID
        abm.AgentID_count += 1 # increment AgentID counter

        aout_t = Vector{Any}(undef, length(abm.p.spc.adata)) #
        
        return a
    end
end

DEBAgent

In [54]:
Vector{Any}(undef, 0)

Any[]

In [ ]:
getpropertagents[1].S

1

In [34]:
"""
Update an agent's reference to global state variables.
"""
function update_glb!(agent::AbstractAgent, abm::AbstractABM)
    map!(x -> x, agent.u.glb, abm.u) 
end

"""
step!(agent::AbstractAgent, abm::AbstractABM; odefuncs::Vector{Function}, rulefuncs::Vector{Function})

Generic definition of agent step. \n
This definition is generic, so that the function body does not have to be modified 
in order to simulate different models.
"""
function step!(agent::AbstractAgent, abm::ABM)
    du, u, p = agent.du, agent.u, agent.p # unpack agent substates, derivatives and parameters
    t = abm.t

    update_glb!(agent, abm) # update references to global state variables
    
    for func! in agent.p.spc.rulefuncs # apply all rule-based functions
        func!(agent, abm)
    end

    for func! in agent.p.spc.odefuncs # apply all ODE-based functions
        func!(du, u, p, t) 
    end


    map!(abm.euler, u.agn, du.agn, u.agn) # apply the euler scheme to agent substates
end

"""
record!(agent::DEBAgent, abm::ABM)

Record agent output (only if `p.glb.recordagentvars == true`).
"""
function record!(agent::AbstractAgent, abm::ABM)
    if abm.p.glb.recordagentvars && isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(abm.aout, (t = abm.t, AgentID = agent.AgentID, u = copy(agent.u.agn)))
    end
end


record!

In [44]:
adata = [:S, :R]
agents = ((S = 1, R = 2), (S = 3, R = 4), (S = 4, R = 5))

((S = 1, R = 2), (S = 3, R = 4), (S = 4, R = 5))

In [35]:

"""
record!(abm::ABM)

Record model-level output.
"""
function record!(abm::ABM)
    if isapprox(abm.t%abm.saveat, 0, atol = abm.dt)
        push!(abm.mout, (t = abm.t, u = copy(abm.u)))
    end
end

"""
Run the ABM.
"""
function run!(abm::AbstractABM)
    while abm.t <= abm.p.glb.t_max
        step!(abm)
    end
end

run!

In [36]:

"""
    prepare_output(abm::AbstractABM)

Prepare ABM output, converting Vectors of ComponentArrays into DataFrames. 
Two separate DataFrames are returned containing global and agent-level state variables, respectively.
"""
function prepare_output(abm::AbstractABM)

    mout = DataFrame(hcat([[x.t] for x in abm.mout]...)', [:t]) |> 
    x-> hcat(x, DataFrame(hcat([m.u for m in abm.mout]...)', extract_colnames(abm.mout[1].u)))
    
    if abm.p.glb.recordagentvars
        aout = DataFrame(hcat([[x.t, x.AgentID] for x in abm.aout]...)', [:t, :AgentID]) |> 
        x-> hcat(x, DataFrame(hcat([a.u for a in abm.aout]...)', extract_colnames(abm.aout[1].u)))
    else
        aout = DataFrame()
    end

    return mout, aout
end


prepare_output

In [37]:
theta = ABMParamCollection(
    glb = GlobalABMParams(AgentType = DEBAgent)
)

Yhat = ABM(theta)

MethodError: MethodError: no method matching copy(::ComponentVector{Float64, SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}, Tuple{Axis{(X_emb = 1, S = 2, S_0 = 3, H = 4, H_b = 5, R = 6, f_X = 7, I_emb = 8, I_p = 9, I = 10, A = 11, M = 12, J = 13, Q = 14, D_G = 15:15, D_M = 16:16, D_A = 17:17, D_R = 18:18, D_h = 19:19, y_G = 20, y_M = 21, y_A = 22, y_R = 23, h_z = 24, h_S = 25, cum_repro = 26, age = 27, cohort = 28, dead = 29, causeofdeath = 30)}}})
You may have intended to import Base.copy

Closest candidates are:
  copy(!Matched::AbstractParamCollection)
   @ Main c:\Users\simon\Documents\Julia\Hydra\src\Params.jl:11
  copy(!Matched::AbstractParams)
   @ Main c:\Users\simon\Documents\Julia\Hydra\src\Params.jl:10
